In [1297]:
import statistics
import queue
from collections import namedtuple
from tqdm import tqdm
import numpy as np 

In [1298]:
Item = namedtuple("Item", ['index', 'value', 'weight', 'value_per_weight'])
Node = namedtuple("Node", ['direction', 'index', 'value', 'weight', 'optimistic_estimate', 'pessimistic_estimate', 'excluded_nodes', 'explored_nodes'])

In [1390]:
round(123.123456, 3)

123.123

In [1356]:
def get_children(current_node, items):
        
    optimistic_value = 0
    pessimistic_value = 0
    total_weight =  MAX_CAPACITY
    excluded_nodes = current_node.excluded_nodes
    explored_nodes = current_node.explored_nodes

    if current_node.direction != 'root':
        filtered_items = [item for item in items if item.index not in excluded_nodes+[current_node.index]]
        next_node = [item for item in filtered_items if item.weight <= current_node.weight and item.index not in explored_nodes]
    else:
        filtered_items = items
        next_node = items
    
    if not next_node: 
        return None, None
        
    for ele in sorted(filtered_items, key=lambda x: x.value_per_weight, reverse=True):
        if total_weight-ele.weight > 0 :
            optimistic_value += ele.value
            total_weight -= ele.weight  
        else:
            optimistic_value += total_weight * ele.value_per_weight
            break
            
    optimistic_value = np.round(optimistic_value, PRECISION)
    # optimistic_value = int(optimistic_value)


    # for ele in sorted(filtered_items, key=lambda x: x.value_per_weight, reverse=False):
    #     if total_weight-ele.weight > 0 :
    #         pessimistic_value += ele.value
    #         total_weight -= ele.weight  
    #     else:
    #         pessimistic_value += total_weight * ele.value_per_weight
    #         break
            
    # pessimistic_value = np.round(pessimistic_value, 1)
    # pessimistic_value = int(pessimistic_value)
    
    next_node = next_node[0]
    
    if current_node.weight-next_node.weight >= 0 :
        left_node = Node('left', next_node.index, current_node.value+next_node.value, current_node.weight-next_node.weight, current_node.optimistic_estimate, current_node.pessimistic_estimate, excluded_nodes, explored_nodes+[next_node.index])
        right_node = Node('right', next_node.index, current_node.value, current_node.weight, optimistic_value, pessimistic_value, excluded_nodes+[next_node.index], explored_nodes+[next_node.index]) #node_estimate-item.value) 
    else:
        left_node = Node('left', next_node.index, -1, current_node.weight-next_node.weight, -1, -1, excluded_nodes, explored_nodes+[next_node.index])
        right_node = Node('right', next_node.index, current_node.value, current_node.weight, optimistic_value, pessimistic_value, excluded_nodes+[next_node.index], explored_nodes+[next_node.index]) #node_estimate-item.value)

    return left_node, right_node

<!-- ## Depth First Search -->

In [1295]:
file_name = 'ks_30_0'

benchmark = {
    'ks_30_0':99798,
    'ks_50_0':142156,
    'ks_200_0':100236,
    'ks_400_0':3967180,
    'ks_1000_0':109899,
    'ks_10000_0':1099893,
}

with open(f"./data/{file_name}", 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
total_items = int(firstLine[0])
MAX_CAPACITY = int(firstLine[1])

items = []

for i in range(1, total_items+1):
    line = lines[i]
    parts = line.split()
    value_per_weight = np.round(int(parts[0])/int(parts[1]), PRECISION)
    items.append(Item(i-1, int(parts[0]), int(parts[1]), value_per_weight))

min_capacity = min([item.weight for item in items])
total_items = len(items)

optimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight >= 0 :
        optimistic_value += item.value
        total_weight -= item.weight  
    else:
        optimistic_value += total_weight * item.value_per_weight
        break

optimistic_value = np.round(optimistic_value, PRECISION)

pessimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=False):
    if total_weight-item.weight >= 0 :
        pessimistic_value += item.value
        total_weight -= item.weight  
    else:
        pessimistic_value += total_weight * item.value_per_weight
        break
        
# pessimistic_value = int(pessimistic_value)     
# min_capacity = MAX_CAPACITY - total_weight + int(total_weight/item.value_per_weight)

total_weight = MAX_CAPACITY
max_items=0
temp_weight=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight >= 0 and item.weight!=temp_weight:
        total_weight -= item.weight  
        max_items+=1
        
    temp_weight = item.weight


total_weight = MAX_CAPACITY
temp_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight >= 0:
        total_weight -= item.weight  
        temp_items+=1

approaching_value = benchmark.get(file_name, -1)


print('max_capacity: ', MAX_CAPACITY)
print('min_capacity: ', min_capacity)
print('optimistic_value: ', optimistic_value)
print('pessimistic_value: ', pessimistic_value)
print('approaching_value: ', approaching_value)
print('total_items: ', total_items)
print('max_items: ', max_items)
print('temp_items: ', temp_items)

## 99798
## 100236

max_capacity:  100000
min_capacity:  10002
optimistic_value:  99999.0
pessimistic_value:  99921.152
approaching_value:  99798
total_items:  30
max_items:  9
temp_items:  9


In [1296]:
items = sorted(items, key=lambda x: (x.value_per_weight, x.weight), reverse=True)
# # items[:10]
items

[Item(index=0, value=90000, weight=90001, value_per_weight=1.0),
 Item(index=1, value=89750, weight=89751, value_per_weight=1.0),
 Item(index=3, value=89500, weight=89501, value_per_weight=1.0),
 Item(index=5, value=89250, weight=89251, value_per_weight=1.0),
 Item(index=7, value=89000, weight=89001, value_per_weight=1.0),
 Item(index=9, value=88750, weight=88751, value_per_weight=1.0),
 Item(index=11, value=88500, weight=88501, value_per_weight=1.0),
 Item(index=13, value=88250, weight=88251, value_per_weight=1.0),
 Item(index=15, value=88000, weight=88001, value_per_weight=1.0),
 Item(index=17, value=87750, weight=87751, value_per_weight=1.0),
 Item(index=19, value=87500, weight=87501, value_per_weight=1.0),
 Item(index=21, value=87250, weight=87251, value_per_weight=1.0),
 Item(index=23, value=87000, weight=87001, value_per_weight=1.0),
 Item(index=25, value=86750, weight=86751, value_per_weight=1.0),
 Item(index=27, value=86500, weight=86501, value_per_weight=1.0),
 Item(index=29, 

In [1190]:
max_value = 0
unexplored_nodes = []
optimal_path = []
optimal_node = Node('dummy', -1, 0, 0, 0, 0, [], [])
loop_counter = 0

# unexplored_nodes = queue.PriorityQueue()

root_item = items[0]
root_node = Node('root', root_item.index, 0, MAX_CAPACITY, optimistic_value, pessimistic_value, [], [])
left_node, right_node = get_children(root_node, items)

unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)

# unexplored_nodes.put((-right_node.estimate, right_node))
# unexplored_nodes.put((-left_node.estimate, left_node))

# unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.direction, x.estimate), reverse=True)
unexplored_nodes

[Node(direction='right', index=3, value=0, weight=11, optimistic_estimate=21.75, pessimistic_estimate=0, excluded_nodes=[3], explored_nodes=[3]),
 Node(direction='left', index=3, value=4, weight=8, optimistic_estimate=21.75, pessimistic_estimate=19.0, excluded_nodes=[], explored_nodes=[3])]

In [1199]:
explore_node = unexplored_nodes.pop()
    # _, explore_node = unexplored_nodes.get()

print('Exploring: ', explore_node)


left_node, right_node = get_children(explore_node, items)

loop_counter += 1

if left_node is not None:
        # continue
    
    print('Max Value: ', max_value)
    print('Left Child: ', left_node)
    print('Right Child: ', right_node)
            
    left_items = len(set(left_node.explored_nodes)) - len(set(left_node.excluded_nodes))
    right_items = len(set(right_node.explored_nodes)) - len(set(right_node.excluded_nodes))

    
    if right_node.optimistic_estimate > max_value: # and right_items <= max_items:
        unexplored_nodes.append(right_node)   
        # unexplored_nodes.put((-right_node.estimate, right_node))
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            optimal_node = right_node
            

    if left_node.weight >= min_capacity: # and left_items <= max_items:
        unexplored_nodes.append(left_node)
        # unexplored_nodes.put((-left_node.estimate, left_node))
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            optimal_node = left_node

   

    # unexplored_nodes = sorted([node for node in unexplored_nodes if node.optimistic_estimate > max_value], key=lambda x: len(set(x.explored_nodes))-len(set(x.excluded_nodes)), reverse=False)

    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.optimistic_estimate/x.pessimistic_estimate), reverse=True)
    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.optimistic_estimate, reverse=False)

    unexplored_nodes = [node for node in unexplored_nodes if node.optimistic_estimate > max_value]

    
    print('Max Value: ', max_value)
    print(unexplored_nodes)
    print('---------------------------------------')

# value=99751, weight=247, optimistic_estimate=99998, pessimistic_estimate=9988

Exploring:  Node(direction='left', index=1, value=10, weight=6, optimistic_estimate=21.75, pessimistic_estimate=0, excluded_nodes=[3, 2], explored_nodes=[3, 2, 1])
Max Value:  19
Left Child:  Node(direction='left', index=0, value=18, weight=2, optimistic_estimate=21.75, pessimistic_estimate=0, excluded_nodes=[3, 2], explored_nodes=[3, 2, 1, 0])
Right Child:  Node(direction='right', index=0, value=10, weight=6, optimistic_estimate=8, pessimistic_estimate=0, excluded_nodes=[3, 2, 0], explored_nodes=[3, 2, 1, 0])
Max Value:  19
[]
---------------------------------------


In [453]:
optimal_node

Node(direction='left', index=1, value=18, weight=2, optimistic_estimate=21.75, pessimistic_estimate=19.0, excluded_nodes=[], explored_nodes=[0, 1])

In [1399]:
file_name = 'ks_1000_0'
PRECISION = 4
benchmark = {
    'ks_30_0':99798,
    'ks_50_0':142156,
    'ks_200_0':100236,
    'ks_400_0':3967180,
    'ks_1000_0':109899,
    'ks_10000_0':1099893,
}

with open(f"./data/{file_name}", 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
total_items = int(firstLine[0])
MAX_CAPACITY = int(firstLine[1])

items = []

for i in range(1, total_items+1):
    line = lines[i]
    parts = line.split()
    value_per_weight = np.round(int(parts[0])/int(parts[1]), PRECISION)
    items.append(Item(i-1, int(parts[0]), int(parts[1]), value_per_weight))

min_capacity = min([item.weight for item in items])
total_items = len(items)

optimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight >= 0 :
        optimistic_value += item.value
        total_weight -= item.weight  
    else:
        optimistic_value += total_weight * item.value_per_weight
        break

optimistic_value = np.round(optimistic_value, PRECISION)

pessimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=False):
    if total_weight-item.weight >= 0 :
        pessimistic_value += item.value
        total_weight -= item.weight  
    else:
        pessimistic_value += total_weight * item.value_per_weight
        break
        
# pessimistic_value = int(pessimistic_value)     
# min_capacity = MAX_CAPACITY - total_weight + int(total_weight/item.value_per_weight)

total_weight = MAX_CAPACITY
max_items=0
temp_weight=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight >= 0 and item.weight!=temp_weight:
        total_weight -= item.weight  
        max_items+=1
        
    temp_weight = item.weight


total_weight = MAX_CAPACITY
temp_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight >= 0:
        total_weight -= item.weight  
        temp_items+=1

approaching_value = benchmark.get(file_name, -1)


print('max_capacity: ', MAX_CAPACITY)
print('min_capacity: ', min_capacity)
print('optimistic_value: ', optimistic_value)
print('pessimistic_value: ', pessimistic_value)
print('approaching_value: ', approaching_value)
print('total_items: ', total_items)
print('max_items: ', max_items)
print('temp_items: ', temp_items)

max_capacity:  100000
min_capacity:  1
optimistic_value:  109907.94
pessimistic_value:  90036.1556
approaching_value:  109899
total_items:  1000
max_items:  103
temp_items:  106


In [1402]:
# sorted(items, key=lambda x: x.value_per_weight, reverse=True)


[Item(index=642, value=3439, weight=3127, value_per_weight=1.0998),
 Item(index=336, value=15347, weight=13958, value_per_weight=1.0995),
 Item(index=519, value=13710, weight=12469, value_per_weight=1.0995),
 Item(index=59, value=6646, weight=6045, value_per_weight=1.0994),
 Item(index=553, value=14781, weight=13446, value_per_weight=1.0993),
 Item(index=115, value=12721, weight=11574, value_per_weight=1.0991),
 Item(index=504, value=9142, weight=8318, value_per_weight=1.0991),
 Item(index=525, value=6064, weight=5517, value_per_weight=1.0991),
 Item(index=801, value=2295, weight=2088, value_per_weight=1.0991),
 Item(index=485, value=11450, weight=10424, value_per_weight=1.0984),
 Item(index=703, value=5247, weight=4777, value_per_weight=1.0984),
 Item(index=394, value=5739, weight=5227, value_per_weight=1.098),
 Item(index=640, value=16636, weight=15151, value_per_weight=1.098),
 Item(index=627, value=303, weight=276, value_per_weight=1.0978),
 Item(index=889, value=438, weight=399, v

In [1398]:

# for fixed_items in range(1, 30, 1):
    
loop_counter = 0
unexplored_nodes = []

max_value = 0
optimal_node = Node('dummy', -1, 0, 0, 0, 0, [], [])
optimal_branch = -1



# unexplored_nodes = queue.PriorityQueue()

items = sorted(items, key=lambda x: (x.value_per_weight, x.weight), reverse=True)


root_item = items[0]
root_node = Node('root', root_item.index, 0, MAX_CAPACITY, optimistic_value, pessimistic_value, [], [])
left_node, right_node = get_children(root_node, items)

unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)

with tqdm() as pbar:
    while len(unexplored_nodes) > 0 and loop_counter < 10000: #not unexplored_nodes.empty(): #len(unexplored_nodes) > 0:
        
        explore_node = unexplored_nodes.pop()
    
        left_node, right_node = get_children(explore_node, items)
    
        if left_node is None:
            continue
    
        loop_counter += 1
        pbar.update(1)
    
        left_items = len(set(left_node.explored_nodes)) - len(set(left_node.excluded_nodes))
        right_items = len(set(right_node.explored_nodes)) - len(set(right_node.excluded_nodes))

        if left_node.weight > min_capacity:# and left_items <= max_items:
            unexplored_nodes.append(left_node)
            # unexplored_nodes.put((-left_node.estimate, left_node))
        else:     
            if left_node.value > max_value:
                max_value = left_node.value
                optimal_node = left_node
                optimal_branch = len(set(optimal_node.explored_nodes)-set(optimal_node.excluded_nodes))
                print(max_value)
                
        if right_node.optimistic_estimate > max_value: # and right_items <= max_items: # and right_node.pessimistic_estimate > optimal_node.pessimistic_estimate:
            unexplored_nodes.append(right_node)   
            # unexplored_nodes.put((-right_node.estimate, right_node))
        else:     
            if right_node.value > max_value: 
                max_value = right_node.value
                optimal_node = right_node
                optimal_branch = len(set(optimal_node.explored_nodes)-set(optimal_node.excluded_nodes))
                print(max_value)

                
        if max_value >= approaching_value:
            break

        
        unexplored_nodes = sorted([node for node in unexplored_nodes if node.optimistic_estimate > max_value and node.weight >= min_capacity], key=lambda x: (x.direction, x.value, x.optimistic_estimate), reverse=True)
        # unexplored_nodes = sorted([node for node in unexplored_nodes if node.optimistic_estimate > max_value and node.weight >= min_capacity], key=lambda x: (x.optimistic_estimate, x.weight, x.value), reverse=False)
        # sorted([node for node in unexplored_nodes if node.optimistic_estimate > max_value and node.weight >= min_capacity ], key=lambda x: (x.optimistic_estimate, x.weight), reverse=True)
        # unexplored_nodes = [node for node in unexplored_nodes if node.optimistic_estimate > max_value]
        # unexplored_nodes = sorted([node for node in unexplored_nodes if node.optimistic_estimate > max_value and node.weight >= min_capacity ], key=lambda x: (x.optimistic_estimate-x.value), reverse=True)

        # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.value, reverse=False)
        # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.direction, x.value, x.optimistic_estimate), reverse=False)
        # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: len(x.explored_nodes)-len(x.excluded_nodes), reverse=True)
        # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.pessimistic_estimate, reverse=True)


max_value

37it [00:00, 125.81it/s]

1099860


64it [00:00, 118.31it/s]

1099864


123it [00:01, 105.35it/s]

1099867


192it [00:01, 105.56it/s]

1099893


1099893

In [ ]:
142418.67875227213-132240

In [1374]:
optimal_node

Node(direction='left', index=164, value=3967180, weight=7, optimistic_estimate=3967416.9066, pessimistic_estimate=3912711.1507, excluded_nodes=[], explored_nodes=[374, 376, 375, 377, 107, 164])

In [ ]:
approaching_value 

In [1393]:
optimal_path = [0]*len(items)

selected_items = set(optimal_node.explored_nodes)-set(optimal_node.excluded_nodes)

for idx in selected_items:
    optimal_path[idx] = 1

In [1396]:
optimal_path)

15

In [1081]:
verify_value = 0
verify_weight = 0
items = sorted(items, key=lambda x: x.index, reverse=False)

for idx, flag in enumerate(optimal_path):
    if flag:
        print(items[idx])
        verify_value += items[idx].value
        verify_weight += items[idx].weight
        

Item(index=59, value=6646, weight=6045, value_per_weight=1.0994210090984284)
Item(index=115, value=12721, weight=11574, value_per_weight=1.0991014342491792)
Item(index=124, value=1007, weight=920, value_per_weight=1.0945652173913043)
Item(index=194, value=1007, weight=924, value_per_weight=1.0898268398268398)
Item(index=210, value=253, weight=231, value_per_weight=1.0952380952380953)
Item(index=336, value=15347, weight=13958, value_per_weight=1.0995128241868461)
Item(index=394, value=5739, weight=5227, value_per_weight=1.097952936674957)
Item(index=485, value=11450, weight=10424, value_per_weight=1.098426707597851)
Item(index=504, value=9142, weight=8318, value_per_weight=1.0990622745852368)
Item(index=506, value=256, weight=234, value_per_weight=1.0940170940170941)
Item(index=519, value=13710, weight=12469, value_per_weight=1.099526826529794)
Item(index=525, value=6064, weight=5517, value_per_weight=1.0991480877288382)
Item(index=627, value=303, weight=276, value_per_weight=1.09782608

In [1082]:
verify_value, verify_weight

(109865, 100000)

In [1083]:
half_items

NameError: name 'half_items' is not defined

In [ ]:
items = sorted(items, key=lambda x: x.weight, reverse=True)
half_items = [item for item in items if item.weight <= MAX_CAPACITY//4]
exclude_indexes = [item.index for item in half_items]
other_half_items = [item for item in items if item.index not in exclude_indexes]
other_half_items = sorted(other_half_items, key=lambda x: x.weight, reverse=False)
custom_sorted_items = half_items+other_half_items
items = custom_sorted_items

In [ ]:
21//4

In [ ]:
MAX_CAPACITY

In [1150]:
# Define the namedtuple structure
Record = namedtuple('Record', ['A', 'B'])

# Create a list of namedtuple instances
records = [
    Record(10, 'x'),
    Record(15, 'y'),
    Record(20, 'x'),
    Record(12, 'y'),
    Record(25, 'z'),
    Record(18, 'x')
]

# Create a dictionary to store the highest value of A for each unique value of B
highest_a_values = {}

# Iterate through the list and update the dictionary
for record in records:
    if record.B in highest_a_values:
        highest_a_values[record.B] = max(highest_a_values[record.B], record.A)
    else:
        highest_a_values[record.B] = record.A

# Filter the list to retain records with the highest value of A for each unique value of B
filtered_records = [record for record in records if record.A == highest_a_values[record.B]]

print(filtered_records)

[Record(A=15, B='y'), Record(A=20, B='x'), Record(A=25, B='z')]
